In [ ]:
import pandas as pd
# 读入 csv 文件中的数据，sep 代表 csv 的分隔符，name 则是代表每一列的字段名
# 返回的是类似二维表的 DataFrame 类型数据 
ratings = pd.read_csv("./ratings.csv",sep=",",names=["user","movie_id","rating"])
movies = pd.read_csv("./movies.csv",sep=",",names=["movie_id","movie_name"])

In [ ]:
ratings.head()

In [ ]:
# 按照用户进行分组统计用户评分的次数
# groupby 按参数指定的字段进行分组，可以是多个字段
# count是对分组后的数据进行计数
# sort_values 则是按照某些字段的值进行排序，ascending=False 代表逆序
ratings_gb_user = ratings.groupby('user').count().sort_values(by='movie_id', ascending=False)

In [ ]:
ratings_gb_user.head()

In [ ]:
ratings_gb_user.shape

In [ ]:
ratings_gb_user.describe()

In [ ]:
ratings_gb_user.movie_id.hist(bins=50)

In [ ]:
# range 代表需要显示的很坐标的取值范围
ratings_gb_user.movie_id.hist(bins=50,range=[1,10])

In [ ]:
# 按照用户进行分组统计用户评分平均值
# groupby 按参数指定的字段进行分组，可以是多个字段
# count是对分组后的数据进行计数
# sort_values 则是按照某些字段的值进行排序，ascending=False 代表逆序
user_rating = ratings.groupby('user').mean().sort_values(by='rating', ascending=False)

In [ ]:
user_rating.rating.describe()

In [ ]:
user_rating.head()

In [ ]:
# merge 函数是类似数据库的关联操作
# how 参数代表关联的方式，例如 inner 是内关联，left 是左关联， right 代表有关联
# on 是关联时使用的键名，由于 ratings 和 movies 对应的电影的字段名是一样的，所以可以写一个，如果不一样则需要使用left_on 和 right_on 参数
ratings_gb_user = pd.merge(ratings_gb_user, user_rating, how='left', on='user')

In [ ]:
ratings_gb_user = ratings_gb_user.rename(columns={'movie_id_x':'movie_id','rating_y':'rating'})

In [ ]:
ratings_gb_user.head()

In [ ]:

# 画散点图，可以指定x轴和y轴
ratings_gb_user.plot(x='movie_id', y='rating', kind='scatter')

In [ ]:
# 按照用户进行分组统计用户评分的次数
# groupby 按参数指定的字段进行分组，可以是多个字段
# count是对分组后的数据进行计数
# sort_values 则是按照某些字段的值进行排序，ascending=False 代表逆序
ratings_gb_movie = ratings.groupby('movie_id').count().sort_values(by='user', ascending=False)

In [ ]:
# merge 函数是类似数据库的关联操作
# how 参数代表关联的方式，例如 inner 是内关联，left 是左关联， right 代表有关联
# on 是关联时使用的键名，由于 ratings 和 movies 对应的电影的字段名是一样的，所以可以写一个，如果不一样则需要使用left_on 和 right_on 参数
ratings_gb_movie = pd.merge(ratings_gb_movie,movies, how='left', on='movie_id')

In [ ]:
ratings_gb_movie.head()

In [ ]:
# 按照用户进行分组统计用户评分平均值
# groupby 按参数指定的字段进行分组，可以是多个字段
# count是对分组后的数据进行计数
# sort_values 则是按照某些字段的值进行排序，ascending=False 代表逆序
movie_rating = ratings.groupby('movie_id').mean().sort_values(by='rating', ascending=False)

In [ ]:
movie_rating.head()

In [ ]:
movie_rating.describe()

In [ ]:
movie_rating.plot(use_index=False)

In [ ]:
ratings_gb_movie = pd.merge(ratings_gb_movie, movie_rating, how='left', on='movie_id')

In [ ]:
ratings_gb_movie.head()

In [ ]:
ratings_gb_movie.plot(x='user',y='rating_y',kind='scatter',figsize=(9,6))

In [ ]:
# 根据余弦相似性建立邻近性矩阵
ratings_pivot=ratings.pivot('user','movie_id','rating')
ratings_pivot.fillna(value=0)
m,n=ratings_pivot.shape
userdist=np.zeros([m,m])
for i in range(m):
    for j in range(m):
        userdist[i,j]=np.dot(ratings_pivot.iloc[i,],ratings_pivot.iloc[j,]) \
        /np.sqrt(np.dot(ratings_pivot.iloc[i,],ratings_pivot.iloc[i,])\
        *np.dot(ratings_pivot.iloc[j,],ratings_pivot.iloc[j,]))
proximity_matrix=pd.DataFrame(userdist,index=list(ratings_pivot.index),columns=list(ratings_pivot.index))

In [ ]:
# 找到临近的 k 个值
def find_user_knn(user, proximity_matrix=proximity_matrix, k=10):
    nhbrs=userdistdf.sort(user,ascending=False)[user][1:k+1]
    #在一列中降序排序，除去第一个（自己）后为近邻
    return nhbrs

In [ ]:
# 获取推荐电影的列表
def recommend_movie(user, ratings_pivot=ratings_pivot, proximity_matrix=proximity_matrix):
    nhbrs=find_user_knn(user, proximity_matrix=proximity_matrix, k=10)
    recommendlist={}
    for nhbrid in nhbrs.index:
        ratings_nhbr=ratings[ratings['user']==nhbrid]
        for movie_id in ratings_nhbr['movie_id']:
            if movie_id not in recommendlist:
                recommendlist[movie_id]=nhbrs[nhbrid]
            else:
                recommendlist[movie_id]=recommendlist[movie_id]+nhbrs[nhbrid]
    # 去除用户已经评分过的电影
	ratings_user =ratings[ratings['user']==user]
    for movie_id in ratings_user['movie_id']:
        if movie_id in recommendlist:
            recommendlist.pop(movie_id)
    output=pd.Series(recommendlist)
    recommendlistdf=pd.DataFrame(output, columns=['score'])
    recommendlistdf.index.names=['movie_id']
    return recommendlistdf.sort('score',ascending=False)